#  IN3050/IN4050 Mandatory Assignment 1: Traveling Salesman Problem


## Rules
Before you begin the exercise, review the rules at this website:
https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html
(This is an individual assignment. You are not allowed to deliver together or copy/share source-code/answers
with others.)

Especially, notice that you are **not allowed to use code or parts of code written by others** in your submission. We do check your code against online repositories, so please be sure to **write all the code yourself**. Read also the "Routines for handling suspicion of cheating and attempted cheating at the University of Oslo": https://www.uio.no/english/studies/examinations/cheating/index.html By submitting this assignment, you confirm that you are familiar with the rules and the consequences of breaking them.

### Delivery

**Deadline**: Friday, February 25 2022, 23:59

Your submission should be delivered in Devilry. You may redeliver in Devilry before the deadline, but include all files in the last delivery, as only the last delivery will be read. You are recommended to upload preliminary versions hours (or days) before the final deadline.

### What to deliver?

You are recommended to solve the exercise in a Jupyter notebook, but you might solve it in a Python program if you prefer.

If you choose Jupyter, you should deliver the notebook. You should answer all questions and explain what you are doing in Markdown. Still, the code should be properly commented. The notebook should contain results of your runs. In addition, you should make a pdf of your solution which shows the results of the runs.

If you prefer not to use notebooks, you should deliver the code, your run results, and a pdf-report where you answer all the questions and explain your work.

Your report/notebook should contain your name and username.

Deliver one single zipped folder (.zip, .tgz or .tar.gz) which contains your complete solution.

Important: if you weren’t able to finish the assignment, use the PDF report/Markdown to elaborate on what you’ve tried and what problems you encountered. Students who have made an effort and attempted all parts of the assignment will get a second chance even if they fail initially. This exercise will be graded PASS/FAIL.

## Introduction
In this exercise, you will attempt to solve an instance of the traveling salesman problem (TSP) using different
methods. The goal is to become familiar with evolutionary algorithms and to appreciate their effectiveness on a
difficult search problem. You may use whichever programming language you like, but we strongly suggest that
you try to use Python, since you will be required to write the second assignment in Python. You must write
your program from scratch (but you may use non-EA-related libraries).


|  &nbsp;   | Barcelona | Belgrade |  Berlin | Brussels | Bucharest | Budapest |
|:---------:|:---------:|:--------:|:-------:|:--------:|:---------:|:--------:|
| Barcelona |     0     |  1528.13 | 1497.61 |  1062.89 |  1968.42  |  1498.79 |
|  Belgrade |  1528.13  |     0    |  999.25 |  1372.59 |   447.34  |  316.41  |
|   Berlin  |  1497.61  |  999.25  |    0    |  651.62  |  1293.40  |  1293.40 |
|  Brussels |  1062.89  |  1372.59 |  651.62 |     0    |  1769.69  |  1131.52 |
| Bucharest |  1968.42  |  447.34  | 1293.40 |  1769.69 |     0     |  639.77  |
|  Budapest |  1498.79  |  316.41  | 1293.40 |  1131.52 |   639.77  |     0    |


<center>Figure 1: First 6 cities from csv file.</center>


## Problem
The traveling salesman, wishing to disturb the residents of the major cities in some region of the world in
the shortest time possible, is faced with the problem of finding the shortest tour among the cities. A tour
is a path that starts in one city, visits all of the other cities, and then returns to the starting point. The
relevant pieces of information, then, are the cities and the distances between them. In this instance of the
TSP, a number of European cities are to be visited. Their relative distances are given in the data file, *european_cities.csv*, found in the zip file with the mandatory assignment.

(You will use permutations to represent tours in your programs. If you use Python, the **itertools** module provides
a permutations function that returns successive permutations, this is useful for exhaustive search)

## Helper code for visualizing solutions

Here follows some helper code that you can use to visualize the plans you generate. These visualizations can **help you check if you are making sensible tours or not**. The optimization algoritms below should hopefully find relatively nice looking tours, but perhaps with a few visible inefficiencies.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from time import time
from itertools import permutations
#Map of Europe
europe_map =plt.imread('map.png')

#Lists of city coordinates
city_coords={"Barcelona":[2.154007, 41.390205], "Belgrade": [20.46,44.79], "Berlin": [13.40,52.52], "Brussels":[4.35,50.85],"Bucharest":[26.10,44.44], "Budapest": [19.04,47.50], "Copenhagen":[12.57,55.68], "Dublin":[-6.27,53.35], "Hamburg": [9.99, 53.55], "Istanbul": [28.98, 41.02], "Kiev": [30.52,50.45], "London": [-0.12,51.51], "Madrid": [-3.70,40.42], "Milan":[9.19,45.46], "Moscow": [37.62,55.75], "Munich": [11.58,48.14], "Paris":[2.35,48.86], "Prague":[14.42,50.07], "Rome": [12.50,41.90], "Saint Petersburg": [30.31,59.94], "Sofia":[23.32,42.70], "Stockholm": [18.06,60.33],"Vienna":[16.36,48.21],"Warsaw":[21.02,52.24]}

In [ ]:
#Helper code for plotting plans
#First, visualizing the cities.
import csv
with open("european_cities.csv", "r") as f:
    data = list(csv.reader(f, delimiter=';'))
    cities = data[0]
    distances = data[1:]
    
fig, ax = plt.subplots(figsize=(10,10))

ax.imshow(europe_map, extent=[-14.56,38.43, 37.697 +0.3 , 64.344 +2.0], aspect = "auto")



# Map (long, lat) to (x, y) for plotting
for city,location in city_coords.items():
    x, y = (location[0], location[1])
    plt.plot(x, y, 'ok', markersize=5)
    plt.text(x, y, city, fontsize=12);

In [ ]:
#A method you can use to plot your plan on the map.
def plot_plan(city_order):
    fig, ax = plt.subplots(figsize=(10,10))
    ax.imshow(europe_map, extent=[-14.56,38.43, 37.697 +0.3 , 64.344 +2.0], aspect = "auto")

    # Map (long, lat) to (x, y) for plotting
    for index in range(len(city_order) -1):
        current_city_coords = city_coords[city_order[index]]
        next_city_coords = city_coords[city_order[index+1]]
        x, y = current_city_coords[0], current_city_coords[1]
        #Plotting a line to the next city
        next_x, next_y = next_city_coords[0], next_city_coords[1]
        plt.plot([x,next_x], [y,next_y])
        
        plt.plot(x, y, 'ok', markersize=5)
        plt.text(x, y, index, fontsize=12);
    #Finally, plotting from last to first city
    first_city_coords = city_coords[city_order[0]]
    first_x, first_y = first_city_coords[0], first_city_coords[1]
    plt.plot([next_x,first_x],[next_y,first_y])
    #Plotting a marker and index for the final city
    plt.plot(next_x, next_y, 'ok', markersize=5)
    plt.text(next_x, next_y, index+1, fontsize=12);
    

def calcDistance(permutation):
    totalDistance = 0
    for index, num in enumerate(permutation):
        start = int(permutation[index])
        finish = int(permutation[(index + 1)%len(permutation)])
        totalDistance += float(distances[start][finish])
    return np.around(totalDistance, 2) 

def numberToCityName(num):
    return cities[num]

def numberArrayToCityNames(array):
    res = []
    for row in array:
        res.append(numberToCityName(int(row)))
        
    return res

In [ ]:
#Example usage of the plotting-method.
plan = list(city_coords.keys()) # Gives us the cities in alphabetic order
print(plan)
#plot_plan(plan)
plot_plan(numberArrayToCityNames([0, 12, 7, 11, 16, 3, 8, 6, 21, 19, 14, 10, 4, 9, 20, 1, 5, 22, 23, 2, 17, 15, 13, 18]))

## Exhaustive Search
First, try to solve the problem by inspecting every possible tour. Start by writing a program to find the shortest
tour among a subset of the cities (say, **6** of them). Measure the amount of time your program takes. Incrementally
add more cities and observe how the time increases. Plot the shortest tours you found using the plot_plan method above, for 6 and 10 cities.

In [ ]:
# Implement the algorithm here
bestDistance = 9999999.9999
bestTravel = []

startTime = time()
for permutation in list(permutations("0123456789")):
    totalDistance = 0
    for index, num in enumerate(permutation):
        start = int(permutation[index - 1])
        finish = int(permutation[index])
        totalDistance += float(distances[start][finish])
        
    if(float(totalDistance) < bestDistance):
        bestDistance = totalDistance
        bestTravel = numberArrayToCityNames(permutation)
finishTime = time()
print(finishTime-startTime)
plot_plan(bestTravel)
print(bestTravel)
print(str(np.around(bestDistance, 2)) + "km")

What is the shortest tour (i.e., the actual sequence of cities, and its length) among the first 10 cities (that is,
the cities starting with B,C,D,H and I)? How long did your program take to find it? Calculate an approximation of how long it would take to perform exhaustive search on all 24 cities?

In [ ]:
# Answer
# The shortest tour for ten cities is
# ['Hamburg', 'Brussels', 'Dublin', 'Barcelona', 'Belgrade', 'Istanbul', 'Bucharest', 'Budapest', 'Berlin', 'Copenhagen']
# with 7486.31 km
# It took 25.395 seconds to calculate it
# My approximation for 24 cities is 138 bilions of years
# I did it multiplicating the previous time with the number of cities (as it was a factorial)

## Hill Climbing
Then, write a simple hill climber to solve the TSP. How well does the hill climber perform, compared to the result from the exhaustive search for the first **10 cities**? Since you are dealing with a stochastic algorithm, you
should run the algorithm several times to measure its performance. Report the length of the tour of the best,
worst and mean of 20 runs (with random starting tours), as well as the standard deviation of the runs, both with the **10 first cities**, and with all **24 cities**. Plot one of the the plans from the 20 runs for both 10 cities and 24 cities (you can use plot_plan). 

In [ ]:
# Implement the algorithm here
numberOfCities = 24
iterationTimes = 50000
totalTries = 20

print("kilometers;seconds")

for i in range(totalTries):
    startTime = time()
    bestPath = np.random.permutation(numberOfCities)
    bestPathKm = calcDistance(bestPath)

    #plot_plan(numberArrayToCityNames(bestPath))
    #print(calcDistance(bestPath))

    for i in range(iterationTimes):
        position = np.random.randint(numberOfCities)
        leftP = position - 1
        rightP = (position + 1)%numberOfCities

        leftPath = list(bestPath)
        rightPath = list(bestPath)

        leftPath[leftP], leftPath[position] = leftPath[position], leftPath[leftP]
        leftPathKm = calcDistance(leftPath)

        rightPath[rightP], rightPath[position] = rightPath[position], rightPath[rightP]
        rightPathKm = calcDistance(rightPath)

        if(rightPathKm < bestPathKm and rightPathKm < leftPathKm):
            bestPath = rightPath
            bestPathKm = rightPathKm
        elif(leftPathKm < bestPathKm and leftPathKm < rightPathKm):
            bestPath = leftPath
            bestPathKm = leftPathKm
    finishTime = time()

    print(str(calcDistance(bestPath)) + ";" + str(finishTime - startTime))
    print(bestPath)
    bestPath = numberArrayToCityNames(bestPath)
    #plot_plan(bestPath)


#perms = permutations

## Genetic Algorithm
Next, write a genetic algorithm (GA) to solve the problem. Choose mutation and crossover operators that are appropriate for the problem (see chapter 4.5 of the Eiben and Smith textbook). Choose three different values for the population size. Define and tune other parameters yourself and make assumptions as necessary (and report them, of course).

For all three variants: As with the hill climber, report best, worst, mean and standard deviation of tour length out of 20 runs of the algorithm (of the best individual of last generation). Also, find and plot the average fitness of the best fit individual in each generation (average across runs), and include a figure with all three curves in the same plot in the report. Conclude which is best in terms of tour length and number of generations of evolution
time.

Finally, plot an example optimized tour (the best of the final generation) for the three different population sizes, using the plot_plan method.

In [ ]:
# Implement the algorithm here
numberOfCities = 10
# 100 300 700
populationSize = -1
sizes = [100, 300, 700]
totalTries = 20
iterations = 500
population = []
offspring = []
#plot_plan(citiesTour)

print("kilometers;seconds")
for indexSize in range (3):
    populationSize = sizes[indexSize]
    offspring = []
    fitnessMean = [0 for i in range(iterations)]
    for tries in range(totalTries):
        population = []
        startTime = time()
        for i in range(populationSize):
            path = np.random.permutation(numberOfCities)
            population.append((list(path), calcDistance(path)))

        citiesTour = numberArrayToCityNames(population[0][0])
        for iterIndex in range(0, iterations):
            offspring = []
            for i in range(0, populationSize, 2):
                child =  [-1]*numberOfCities
                fatherOriginal = list(population[i][0])
                father = list(fatherOriginal)
                motherOriginal = list(population[i+1][0])
                mother = list(motherOriginal)

                #mutation
                start = np.random.randint(0, numberOfCities)
                finish = np.random.randint(0, numberOfCities)
                if(start > finish):
                    start, finish = finish, start
                father[start], father[finish] = father[finish], father[start]

                start = np.random.randint(0, numberOfCities)
                finish = np.random.randint(0, numberOfCities)
                if(start > finish):
                    start, finish = finish, start
                mother[start], mother[finish] = mother[finish], mother[start]

                #recombination
                start = np.random.randint(0, numberOfCities)
                finish = np.random.randint(0, numberOfCities)
                if(start > finish):
                    start, finish = finish, start

                child[start:finish+1] = father[start:finish+1]

                notCopied = [] #c'è nel figlio, copiato dal padre, ma non nella madre in start:finish+1 range
                for motherAllele in mother[start:finish+1]:
                    if(np.where(father[start:finish+1] == motherAllele)[0].size <= 0):
                        notCopied.append(motherAllele)


                for aNotCopy in notCopied:
                    isCopied = False
                    motherPosition = np.where(mother == aNotCopy)[0][0]
                    fatherValue = father[motherPosition]
                    while not isCopied:
                        motherPosition = np.where(mother == fatherValue)[0][0]
                        fatherValue = father[motherPosition]
                        if(child[motherPosition] == -1):
                            child[motherPosition] = aNotCopy
                            isCopied = True

                for i, childAllele in enumerate(child):
                    if(childAllele == -1):
                        child[i] = mother[i]

                offspring.append((list(child), calcDistance(child)))
            population = population + offspring
            population = sorted(population, key=lambda i: i[-1])
            population = population[0:populationSize]
            fitnessMean[iterIndex] = fitnessMean[iterIndex] + np.mean(population[0][1])
        finishTime = time()
        #print(population[0][1])
        #print(str(population[0][0]))
        print(str(population[0][1]) + ";" + str(finishTime - startTime) + ";" + str(population[0][0]))

    for divideMean in range(iterations):
            fitnessMean[divideMean] = fitnessMean[divideMean] / totalTries
    print("")
    y = np.array(fitnessMean)
    x = np.arange(0, iterations)
    #plt.plot(x,y)
    plt.plot(x,y, label = ("population " + str(sizes[indexSize])))
    #plt.show()
#plt.legend()
plt.show()
citiesTour = numberArrayToCityNames(population[0][0])
#plot_plan(citiesTour)

Among the first 10 cities, did your GA find the shortest tour (as found by the exhaustive search)? Did it come close? 

For both 10 and 24 cities: How did the running time of your GA compare to that of the exhaustive search? 

How many tours were inspected by your GA as compared to by the exhaustive search?

In [ ]:
# Answer
# Yes, it has found the shortest tour
# About the time, using 700 "people" and 500 generations, it was 8 seconds for 10 cities against 25 seconds
# and 13 seconds against 138 bilion years (estimated) for 24 cities

# Exaustive search inspect n! permutations

# GA inspect populationSize+(childrenSize*iterations)
# where childrenSize is populationSize/2

# So, if we have 24 cities, 700 people, 500 generations
# Exaustive = 24! = 620,448,401,733,239,000,000,000
# GA = 700 + ((700/2)*500) = 175,700

# The best result was with a big population. I think it is because more people means more genetic diversity,
# so it's more probable to find someone better.

## Hybrid Algorithm (IN4050 only)
### Lamarckian
Lamarck, 1809: Traits acquired in parents’ lifetimes can be inherited by offspring. In general the algorithms are referred to as Lamarckian if the result of the local search stage replaces the individual in the population.
### Baldwinian
Baldwin effect suggests a mechanism whereby evolutionary progress can be guided towards favourable adaptation without the changes in individual's fitness arising from learning or development being reflected in changed genetic characteristics. In general the algorithms are referred to as Baldwinian if the original member is kept, but has as its fitness the value belonging to the outcome of the local search process.


(See chapter 10 and 10.2.1 from Eiben and Smith textbook for more details. It will also be lectured in Lecure 4)

### Task
Implement a hybrid algorithm to solve the TSP: Couple your GA and hill climber by running the hill climber a number of iterations on each individual in the population as part of the evaluation. Test both Lamarckian and Baldwinian learning models and report the results of both variants in the same way as with the pure GA (min,
max, mean and standard deviation of the end result and an averaged generational plot). How do the results compare to that of the pure GA, considering the number of evaluations done?

In [ ]:
# Implement algorithm here